# Acquire

### 1. build the dataset

In [1]:
import pandas as pd
import numpy as np
import re
from requests import get
import os
from bs4 import BeautifulSoup

> Because the topics page has dynamic content (which loads more results each time you click a button) we clickd the button a whole bunch and then downloaded the source code as a text file. That text file is our soup.

In [2]:
repos = []
with open('repo_source.txt') as f:
    source_text = f.read()
    
soup = BeautifulSoup(source_text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <link href="https://github.githubassets.com" rel="dns-prefetch"/>
  <link href="https://avatars0.githubusercontent.com" rel="dns-prefetch"/>
  <link href="https://avatars1.githubusercontent.com" rel="dns-prefetch"/>
  <link href="https://avatars2.githubusercontent.com" rel="dns-prefetch"/>
  <link href="https://avatars3.githubusercontent.com" rel="dns-prefetch"/>
  <link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
  <link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
  <link crossorigin="anonymous" href="https://github.githubassets.com/assets/frameworks-bde286da8a2da653462f03445b922de3.css" integrity="sha512-veKG2ootplNGLwNEW5It4+lJXwpdOVR1Ry4MBDNvdgL6OgBkiAXaZAZnIKkw6sIygnfGqQhR5q7UdS1llqd6/g==" media="all" rel="stylesheet">
   <link crossorigin="anonymous" href="https://github.githubassets.com/assets/github-88e136c88e1f96ad0c830221824e2d82.css" integrity="sha512-iOE2yI4

In [3]:
soup.find_all('h1', class_="f3 text-gray text-normal lh-condensed")[0].text

'\n\n            CSSEGISandData\n\n\n          /\n          \n            COVID-19\n\n'

In [4]:
soup.find_all('h1', class_="f3 text-gray text-normal lh-condensed")[1].text

'\n\n            tokyo-metropolitan-gov\n\n\n          /\n          \n            covid19\n\n'

In [5]:
soup.find_all('h1', class_="f3 text-gray text-normal lh-condensed")[2].text

'\n\n            nytimes\n\n\n          /\n          \n            covid-19-data\n\n'

In [6]:
soup.find_all('h1', class_="f3 text-gray text-normal lh-condensed")[3].text

'\n\n            covid19india\n\n\n          /\n          \n            covid19india-react\n\n'

In [7]:
link_list = soup.find_all('h1', class_="f3 text-gray text-normal lh-condensed")
link_list

[<h1 class="f3 text-gray text-normal lh-condensed">
 <a data-ga-click="Explore, go to repository owner, location:explore feed" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"OWNER","click_visual_representation":"REPOSITORY_OWNER_HEADING","actor_id":60628828,"record_id":60674295,"originating_url":"https://github.com/topics/covid-19?o=desc&amp;s=stars","user_id":60628828}}' data-hydro-click-hmac="9933bf4bac76c107f0945e041ce49046dd3ecc1abe49510990c1a6e523ff4c0a" href="/CSSEGISandData">
             CSSEGISandData
 </a>
 
           /
           <a class="text-bold" data-ga-click="Explore, go to repository, location:explore feed" data-hydro-click='{"event_type":"explore.click","payload":{"click_context":"REPOSITORY_CARD","click_target":"REPOSITORY","click_visual_representation":"REPOSITORY_NAME_HEADING","actor_id":60628828,"record_id":238316428,"originating_url":"https://github.com/topics/covid-19?o=desc&amp;s=stars","user_id":6

In [8]:
for link in link_list:
    repos.append(link.text)

In [9]:
repos

['\n\n            CSSEGISandData\n\n\n          /\n          \n            COVID-19\n\n',
 '\n\n            tokyo-metropolitan-gov\n\n\n          /\n          \n            covid19\n\n',
 '\n\n            nytimes\n\n\n          /\n          \n            covid-19-data\n\n',
 '\n\n            covid19india\n\n\n          /\n          \n            covid19india-react\n\n',
 '\n\n            pcm-dpc\n\n\n          /\n          \n            COVID-19\n\n',
 '\n\n            ieee8023\n\n\n          /\n          \n            covid-chestxray-dataset\n\n',
 '\n\n            WorldHealthOrganization\n\n\n          /\n          \n            app\n\n',
 '\n\n            ahmadawais\n\n\n          /\n          \n            corona-cli\n\n',
 '\n\n            ExpDev07\n\n\n          /\n          \n            coronavirus-tracker-api\n\n',
 '\n\n            neherlab\n\n\n          /\n          \n            covid19_scenarios\n\n',
 '\n\n            geohot\n\n\n          /\n          \n            coro

In [10]:
len(repos)

30

In [11]:
regexp = r'''
(?P<team_name>[^\n\s]*)
\n*\s*
(?P<sep>\/)
\n*\s*
(?P<repo_name>[^\n\s].+)\n\n
'''
match = re.search(regexp, repos[0], re.VERBOSE)
match.groupdict()

{'team_name': 'CSSEGISandData', 'sep': '/', 'repo_name': 'COVID-19'}

In [12]:
df = pd.DataFrame(repos, columns=['original'])
df = df.original.str.extract(regexp, re.VERBOSE)
df['repos'] = '"' + df.team_name + df.sep + df.repo_name +  '"'
df

,team_name,sep,repo_name,repos
0,CSSEGISandData,/,COVID-19,"""CSSEGISandData/COVID-19"""
1,tokyo-metropolitan-gov,/,covid19,"""tokyo-metropolitan-gov/covid19"""
2,nytimes,/,covid-19-data,"""nytimes/covid-19-data"""
3,covid19india,/,covid19india-react,"""covid19india/covid19india-react"""
4,pcm-dpc,/,COVID-19,"""pcm-dpc/COVID-19"""
5,ieee8023,/,covid-chestxray-dataset,"""ieee8023/covid-chestxray-dataset"""
6,WorldHealthOrganization,/,app,"""WorldHealthOrganization/app"""
7,ahmadawais,/,corona-cli,"""ahmadawais/corona-cli"""
8,ExpDev07,/,coronavirus-tracker-api,"""ExpDev07/coronavirus-tracker-api"""
9,neherlab,/,covid19_scenarios,"""neherlab/covid19_scenarios"""


In [13]:
REPOS = pd.Series(df.repos)
REPOS

0                       "CSSEGISandData/COVID-19"
1                "tokyo-metropolitan-gov/covid19"
2                         "nytimes/covid-19-data"
3               "covid19india/covid19india-react"
4                              "pcm-dpc/COVID-19"
5              "ieee8023/covid-chestxray-dataset"
6                   "WorldHealthOrganization/app"
7                         "ahmadawais/corona-cli"
8              "ExpDev07/coronavirus-tracker-api"
9                    "neherlab/covid19_scenarios"
10                                "geohot/corona"
11          "soroushchehresa/awesome-coronavirus"
12                      "mhdhejazi/CoronaTracker"
13                               "pomber/covid19"
14                     "github/covid19-dashboard"
15                            "datasets/covid-19"
16                    "chrismattmann/tika-python"
17           "ImperialCollegeLondon/covid19model"
18                           "owid/covid-19-data"
19                         "lindawangg/COVID-Net"


In [14]:
def get_all_repos(text_file):
    
    # Read in the source code as a text string
    with open(f'{text_file}') as f:
        source_text = f.read()
    
    # Turn the source code string into soup
    soup = BeautifulSoup(source_text, 'html.parser')
    
    # Look through the soup to find each repo name
    link_list = soup.find_all('h1', class_="f3 text-gray text-normal lh-condensed")
    
    # Start an empty list to hold the repos
    raw_repos = []
    
    # Add each repo to repo list
    for link in link_list:
        raw_repos.append(link.text)
    
    # Look through the repo list to find the cleaned up repo name
    regexp = r'''
    (?P<team_name>[^\n\s]*)
    \n*\s*
    (?P<sep>\/)
    \n*\s*
    (?P<repo_name>[^\n\s].+)\n\n
    '''
    
    # Make a df to handle the formatting of each repo name
    df = pd.DataFrame(raw_repos, columns=['original'])
    df = df.original.str.extract(regexp, re.VERBOSE)
    df['repos'] = '"' + df.team_name + df.sep + df.repo_name +  '"'
    
    # Save the formatted repo names as list
    repos = pd.Series(df.repos).tolist()
    
    return repos

In [15]:
import acquire

REPOS = acquire.get_all_repos('repo_source')
REPOS

['CSSEGISandData/COVID-19',
 'tokyo-metropolitan-gov/covid19',
 'nytimes/covid-19-data',
 'covid19india/covid19india-react',
 'pcm-dpc/COVID-19',
 'ieee8023/covid-chestxray-dataset',
 'WorldHealthOrganization/app',
 'ahmadawais/corona-cli',
 'ExpDev07/coronavirus-tracker-api',
 'neherlab/covid19_scenarios',
 'geohot/corona',
 'soroushchehresa/awesome-coronavirus',
 'mhdhejazi/CoronaTracker',
 'pomber/covid19',
 'github/covid19-dashboard',
 'datasets/covid-19',
 'chrismattmann/tika-python',
 'ImperialCollegeLondon/covid19model',
 'owid/covid-19-data',
 'lindawangg/COVID-Net',
 'globalcitizen/2019-wuhan-coronavirus-data',
 'UCSD-AI4H/COVID-CT',
 'javieraviles/covidAPI',
 'MohGovIL/hamagen-react-native',
 'phildini/stayinghomeclub',
 'AaronWard/covidify',
 'GuangchuangYu/nCov2019',
 'covidatlas/coronadatascraper',
 'ryansmcgee/seirsplus',
 'lispczz/pneumonia']

In [16]:
df = acquire.scrape_github_data('repo_source')

In [18]:
df = pd.DataFrame.from_dict(df)
df

,repo,language,readme_contents
0,CSSEGISandData/COVID-19,None,# COVID-19 Data Repository by the Center for S...
1,tokyo-metropolitan-gov/covid19,Vue,# 東京都 新型コロナウイルス感染症対策サイト\n\n![](https://github....
2,nytimes/covid-19-data,None,# Coronavirus (Covid-19) Data in the United St...
3,covid19india/covid19india-react,JavaScript,"<p align=""center"">\n<img src=""https://lh3.goog..."
4,pcm-dpc/COVID-19,None,"<img src=""http://opendatadpc.maps.arcgis.com/s..."
5,ieee8023/covid-chestxray-dataset,Jupyter Notebook,\n#### 🛑 Note: please do not claim diagnostic ...
6,WorldHealthOrganization/app,Dart,None
7,ahmadawais/corona-cli,JavaScript,"<h4 align=""center"">\n <a href=""https://gith..."
8,ExpDev07/coronavirus-tracker-api,Python,"<h1 align=""center"">\n Coronavirus Tracker A..."
9,neherlab/covid19_scenarios,TypeScript,"<h1 id=""covid19_scenarios"" align=""center"">\n ..."
